## 전처리: 엑셀 file에서 질문데이터 로드

In [62]:
import pandas as pd
df=pd.read_excel('simple.xlsx')
df.head()

,group,state,content,next
0,0,질문시작,아코봇을 불러줘,1
1,1,전체모집요강,전체 모집 요강이 뭐야?,6
2,1,수시,수시전형 알려줘,2
3,1,정시,정시전형 알려줘,3
4,1,모집인원,모집 인원이 몇 명이지?,6


In [63]:
#질문 관련 컬럼만 로드
df=df['content']
df.head()

0         아코봇을 불러줘
1    전체 모집 요강이 뭐야?
2         수시전형 알려줘
3         정시전형 알려줘
4    모집 인원이 몇 명이지?
Name: content, dtype: object

In [64]:
#단락구분 등 공백 제거
contents=[]
for i in df:
    i=' '.join(i.split())
    contents.append(i)
    df=pd.DataFrame(contents)
df.head()

,0
0,아코봇을 불러줘
1,전체 모집 요강이 뭐야?
2,수시전형 알려줘
3,정시전형 알려줘
4,모집 인원이 몇 명이지?


In [65]:
#텍스트파일 작성
f = open("df.txt", 'wt', encoding='utf-8')
for i in  range(0, 29):
    data = df.loc[i].to_string()
    f.write(data+'\n')
f.close()

# Soynlp 사용 (Unsupervised Leraning 기반)

In [66]:
#필요한 각종 library 임포트
import urllib.request
from soynlp import DoublespaceLineCorpus
from soynlp.word import WordExtractor


In [67]:
#텍스트 파일로부터 각 line을 문서로 분리
corpus = DoublespaceLineCorpus("df.txt")
len(corpus)

29

In [68]:
#corpus를 학습시키기
word_extractor = WordExtractor()
word_extractor.train(corpus)
word_score_table = word_extractor.extract()

training was done. used memory 0.206 Gb 0.206 Gb
all cohesion probabilities was computed. # words = 9
all branching entropies was computed # words = 42
all accessor variety was computed # words = 42


# 문장 넣어 토큰화하기

### 방법1: LTokenizer는 여러 가지 길이의 L 토큰의 점수를 비교하여 가장 점수 가 높은 L단어를 찾는다

In [69]:
from soynlp.tokenizer import LTokenizer

scores = {word:score.cohesion_forward for word, score in word_score_table.items()}
l_tokenizer = LTokenizer(scores=scores)
l_tokenizer.tokenize("정시 전형을 자세히 알려줄래?", flatten=False) #띄어쓰기 필수!
#l_tokenizer.tokenize("정시전형자세히알려줄래?", flatten=False)

[('정시', ''), ('전형', '을'), ('자세히', ''), ('알려', '줄래?')]

### 방법2: MaxScoreTokenizer는 띄어쓰기가 되지 않는 문장에서 점수가 높은 글자 시퀀스를 순차적으로 찾아내는 토크나이저

In [70]:
from soynlp.tokenizer import MaxScoreTokenizer

maxscore_tokenizer = MaxScoreTokenizer(scores=scores)
#maxscore_tokenizer.tokenize("정시 전형을 자세히 알려줄래?")
maxscore_tokenizer.tokenize("정시전형자세히알려줄래?")

['정시', '전형', '자세히', '알려', '줄래?']